In [1]:
import pandas as pd
import re
import string
from collections import Counter
import matplotlib.pyplot as plt


In [2]:
def remove_numbers(text):
    return " ".join(re.sub(r"[0-9]", " ", text).split())

def remove_unprintable_(text):
    printable = set(string.printable + "ñáéíóúüÑÁÉÍÓÚÜ")
    return "".join(filter(lambda x: x in printable, text))

def remove_punctuation(text):
    pattern = re.compile(r"[^\w\sáéíóúüñÁÉÍÓÚÜÑ]")
    return re.sub(" +", " ", pattern.sub(" ", text))

def reduce_spam(text):
    text = re.sub(r"(\w+)(\s+\1){2,}", r"\1", text)
    text = re.sub(r"(\w+\s+\w+)(\s+\1){2,}", r"\1", text)
    return text

def remove_vowels_accents(text):
    return (
        text.replace("á", "a").replace("é", "e")
            .replace("í", "i").replace("ó", "o")
            .replace("ú", "u").replace("ü", "u")
    )

def remove_stopwords(text, stopwords_list):
    return " ".join([word for word in text.split() if word not in stopwords_list])

def clean_text(text, stopwords_list):
    text = text.lower()
    text = remove_numbers(text)
    text = remove_unprintable_(text)
    text = remove_punctuation(text)
    text = reduce_spam(text)
    text = remove_stopwords(text, stopwords_list)
    text = remove_vowels_accents(text)
    return text.strip()


In [3]:
# Carga del corpus desde Excel (ajusta el path)
df = pd.read_excel("01_bbdd_think_tanks.xlsx")

# Filtrado de fechas (si aplica)
df['FechaPublicación'] = pd.to_datetime(df['FechaPublicación'], errors='coerce')
df = df.dropna(subset=['Corpus'])

# Definición inicial de stopwords vacías (solo para limpieza léxica)
stopwords_list = []

# Aplicar limpieza
df['Corpus_Limpio'] = df['Corpus'].apply(lambda x: clean_text(str(x), stopwords_list))


In [6]:
# Tokenización global del corpus
tokens = " ".join(df["Corpus_Limpio"]).split()

# Conteo de frecuencia
frecuencias = Counter(tokens)
frecuencia_df = pd.DataFrame(frecuencias.items(), columns=["Palabra", "Frecuencia"])
frecuencia_df = frecuencia_df.sort_values(by="Frecuencia", ascending=False).reset_index(drop=True)

# Visualizar top-N palabras
frecuencia_df.head(100)  # Puedes ajustar el N según necesidad


,Palabra,Frecuencia
0,de,1148144
1,la,702622
2,que,538062
3,en,493543
4,el,487584
...,...,...
95,derecho,12704
96,dos,12617
97,durante,12539
98,proceso,12493


In [ ]:
def buscar_palabra_en_documentos(palabra_objetivo, dataframe, columna_corpus):
    palabra_objetivo = palabra_objetivo.lower()
    documentos = dataframe[dataframe[columna_corpus].str.contains(rf"\b{palabra_objetivo}\b", na=False, case=False)]
    return documentos

# Ejemplo de uso
palabra_objetivo = "economía"  # Cambia esta palabra según lo que quieras buscar
documentos_con_palabra = buscar_palabra_en_documentos(palabra_objetivo, df, "Corpus_Limpio")

# Mostrar los documentos encontrados
print(f"Documentos que contienen la palabra '{palabra_objetivo}':")
print(documentos_con_palabra[["Título", "FechaPublicación", "Corpus"]])  # Ajusta las columnas según tu DataFrame

In [7]:
frecuencia_df.to_csv("frecuencia_palabras.csv", index=False)


In [10]:
import pandas as pd
import spacy

# --- Cargar modelo grande de spaCy ---
nlp = spacy.load("es_core_news_lg")

# --- Leer palabras más frecuentes ---
df = pd.read_csv("frecuencia_palabras.csv")
top_words = df.head(1000)["Palabra"].dropna().astype(str).tolist()
text = " ".join(top_words)
doc = nlp(text)

# --- A. POS estructurales vacíos ---
tags_pos_funcionales = {"DET", "ADP", "PRON", "CCONJ", "SCONJ", "AUX", "PART"}

stopwords_pos = {
    token.lemma_.lower()
    for token in doc
    if token.pos_ in tags_pos_funcionales
}

# --- B. Funciones sintácticas vacías (dependencias) ---
deps_funcionales = {"aux", "det", "mark", "cc", "case", "cop", "punct"}

stopwords_dep = {
    token.lemma_.lower()
    for token in doc
    if token.dep_ in deps_funcionales
}

# --- C. Verbos funcionales (light verbs conocidos en español) ---
verbos_funcionales = {
    "hacer", "realizar", "tener", "haber", "decir", "dar", "llevar",
    "ser", "estar", "poner", "tomar", "ver", "pasar"
}

stopwords_verbos = {
    token.lemma_.lower()
    for token in doc
    if token.pos_ == "VERB" and token.lemma_.lower() in verbos_funcionales
}

# --- D. Consolidar todas las stopwords ---
stopwords_total = sorted(stopwords_pos.union(stopwords_dep).union(stopwords_verbos))

# --- Guardar a archivo ---
with open("stopwords_expandida.txt", "w", encoding="utf-8") as f:
    for palabra in stopwords_total:
        f.write(palabra + "\n")

print(f"Stopwords ampliadas generadas: {len(stopwords_total)} palabras")


Stopwords ampliadas generadas: 98 palabras
